# Movie Recommendation System

1. Demographic Filtering (인구 통계학적 필터링)
1. Content Based Filtering (컨텐츠 기반 필터링)
1. Collaborative Filtering (협업 필터링)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dp = '/content/drive/MyDrive/movie_reco_data/'

## 1. Demogrphic Filtering (인구 통계학적 필터링)

In [ ]:
import pandas as pd
import numpy as numpy

df1  = pd.read_csv(dp + 'tmdb_5000_credits.csv')
df2  = pd.read_csv(dp + 'tmdb_5000_movies.csv')

### 데이터 merge

In [ ]:
df1.head()

In [ ]:
df2.head(3)

In [ ]:
df1.shape, df2.shape

In [ ]:
df1['title'].equals(df2['title'])

In [ ]:
df1.columns

In [ ]:
df1.columns = ['id', 'title', 'cast', 'crew']
df1.columns

In [ ]:
df1[['id', 'cast', 'crew']]

In [ ]:
df2 = df2.merge(df1[['id', 'cast', 'crew']], on='id')
df2.head(3)

영화 1 : 영화의 평점이 10/10 -> 5명이 평가

영화 2 : 영화의 평점이8/10 -> 500명이 평가

In [ ]:
C = df2['vote_average'].mean()
C

In [ ]:
m = df2['vote_count'].quantile(0.9)
m

In [ ]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape

In [ ]:
q_movies['vote_count'].sort_values()

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (m + v) * C)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)  # axis = 1 : row단위로 score를 구해서 적용
q_movies.head(3)

In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

In [ ]:
pop= df2.sort_values('popularity', ascending=False)
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.barh(pop['title'].head(10),pop['popularity'].head(10), align='center',
        color='skyblue')
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")

## 2. Content Based Filtering (컨텐츠 기반 필터링)

In [ ]:
df2['overview'].head(5)

Bag of Words - BOW

문장1 : I am a boy

문장2 : I am a girl

I(2), am(2), a(2), boy(1), girl(1)


         I     am    a    boy    girl
 문장1   1     1     1     1      0    (1, 1, 1, 1, 0)
 (I am a boy)

 문장2   1     1     1     0      1    (1, 1, 1, 0, 1)
 (I am a girl)

 피쳐 벡터화.

 문서 100개
 모든 문서에서 나온 단어 10,000 개
 100 * 10,000 = 100만

        단어1,  단어2,  단어3,  단어4,  .....  단어10000
문서1     1       1       3       0
문서2
문서3
..
문서100

1. TfidfVectorizer  (TF-IDF 기반의 벡터화)
2. CounterVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS

In [ ]:
df2['overview'].isnull().values.any()

In [ ]:
df2['overview'] = df2['overview'].fillna('')

In [ ]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])
tfidf_matrix.shape

In [ ]:
tfidf_matrix

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

In [ ]:
cosine_sim.shape

In [ ]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()
indices

In [ ]:
df2.iloc[[3]]

### 추천 영화 10개를 뽑는 함수

In [ ]:
# 영화의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 영화 목록 반환
def get_recommendations(title, cosine_sim=cosine_sim):
    # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
    idx = indices[title]

    # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 코사인 유사도 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)

     # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
    sim_scores = sim_scores[1:11]

    # 추천 영화 목록 10개의 인덱스 정보 추출
    movie_indices = [i[0] for i in sim_scores]

    # 인덱스 정보를 통해 영화 제목 추출
    return df2['title'].iloc[movie_indices]

In [ ]:
test_idx = indices['The Dark Knight Rises']  # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
test_idx

In [ ]:
cosine_sim[3]

In [ ]:
test_sim_scores = list(enumerate(cosine_sim[3]))  # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx에 해당하는 데이터를 (idx, 유사도) 형태로 얻기

In [ ]:
test_sim_scores = sorted(test_sim_scores, key=lambda x : x[1], reverse=True) # 코사인 유사도 기준으로 내림차순 정렬
test_sim_scores[1:11]  # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱

In [ ]:
# 추천 영화 목록 10개의 인덱스 정보 추출
test_movie_indices = [i[0] for i in test_sim_scores[1:11]]
test_movie_indices

In [ ]:
# 인덱스 정보를 통해 영화 제목 추출
df2['title'].iloc[test_movie_indices]

In [ ]:
df2['title'][:20]

In [ ]:
get_recommendations('Avengers: Age of Ultron')

In [ ]:
get_recommendations('The Avengers')

### 다양한 요소 기반 추천 (장르, 감독, 키워드 등)

In [ ]:
df2.head(3)

In [ ]:
df2.loc[0, 'genres']

In [ ]:
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [ ]:
df2.loc[0, 'crew']

In [ ]:
import numpy as np
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
df2['director'] = df2['crew'].apply(get_director)
df2['director']

In [ ]:
df2[df2['director'].isnull()]

In [ ]:
len(df2.loc[0, 'cast'])

In [ ]:
df2.loc[0, 'genres']

In [ ]:
df2.loc[0, 'keywords']

In [ ]:
# 처음 3개의 데이터 중에서 name 에 해당하는 values만 추출
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [ ]:
df2['cast']

In [ ]:
features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

In [ ]:
features = ['director', 'cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)
df2['soup']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])
count_matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])
indices

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
get_recommendations('Up', cosine_sim2)

In [ ]:
get_recommendations('The Martian', cosine_sim2)

In [ ]:
indices['The Martian']

In [ ]:
df2.loc[270]

In [ ]:
df2.loc[4]

In [ ]:
get_recommendations('The Avengers', cosine_sim2)

In [ ]:
import pickle

In [ ]:
movies = df2[['id', 'title']].copy()
movies.head(5)

In [ ]:
pickle.dump(movies, open(dp + 'movies.pickle', 'wb'))

In [ ]:
pickle.dump(cosine_sim2, open(dp + 'cosine_sim.pickle', 'wb'))